<a href="https://colab.research.google.com/github/RajeshworM/Customer-Segmentation-Analysis/blob/main/fasal_panel_district.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
# Step 1: Install necessary packages
!pip install linearmodels

In [84]:
# Step 2: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [85]:
# Step 3: Set the file path (replace with your file path)
file_path = '/content/drive/My Drive/panel_data_2000_2024.csv'
data = pd.read_csv(file_path)
data.head()

,state,year,district,stateid,districtid,area,yield,prod,irrigation_nia,irrigation_canal,...,res_5,res_6,res_7,res_8,res_9,price_crop10m11,price_crop11m11,price_crop12m11,msp11,groundwater
0,Madhya Pradesh,2000,Rewa,7,1,134789.0,0.55,73982.0,82851.0,15467.0,...,0.0,0.26,0.26,0.00,0.00,1563.0,1548.0,1567.0,510,10.24
1,Madhya Pradesh,2001,Rewa,7,1,145397.0,0.94,136581.0,90393.0,17069.0,...,0.0,0.09,0.36,0.29,0.00,1606.0,1601.0,1575.0,530,11.64
2,Madhya Pradesh,2002,Rewa,7,1,140685.0,0.68,96251.0,90415.0,14318.0,...,0.0,0.35,0.36,0.28,0.00,1588.0,1587.0,1577.0,550,10.57
3,Madhya Pradesh,2003,Rewa,7,1,147133.0,0.94,138215.0,87179.0,14795.0,...,0.0,0.35,0.36,0.28,0.00,1642.0,1610.0,1552.0,550,11.20
4,Madhya Pradesh,2004,Rewa,7,1,141563.0,0.63,89532.0,87766.0,13910.0,...,0.0,0.26,0.26,0.28,0.64,1617.0,1611.0,1612.0,560,11.29


In [62]:
# Step 4: Load the data (including independent variables for 2021 to 2024)
import pandas as pd
from linearmodels.panel import RandomEffects



In [78]:
# Step 5: Set the panel index (stateid, districtid, year)
data = data.set_index(['stateid', 'districtid', 'year'])

KeyError: "None of ['stateid', 'districtid', 'year'] are in the columns"

In [64]:
# Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

In [65]:
# Step 6: Calculate the 1-year lag for the average price using .loc
data_sample['avg_price'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1)

# Apply the shift with .loc to modify the DataFrame
data_sample.loc[:, 'avg_price_lag1'] = data_sample.groupby(['stateid', 'districtid'])['avg_price'].shift(1)


<ipython-input-65-e98f3c68a5e4>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1)
<ipython-input-65-e98f3c68a5e4>:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample.loc[:, 'avg_price_lag1'] = data_sample.groupby(['stateid', 'districtid'])['avg_price'].shift(1)


In [66]:
# Step 7: Generate variables by dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
data_sample['msp11_fert2'] = data_sample['msp11'] / data_sample['fert2']
data_sample['msp11_fert3'] = data_sample['msp11'] / data_sample['fert3']

data_sample['avg_price_fert1'] = data_sample['avg_price_lag1'] / data_sample['fert1']
data_sample['avg_price_fert2'] = data_sample['avg_price_lag1'] / data_sample['fert2']
data_sample['avg_price_fert3'] = data_sample['avg_price_lag1'] / data_sample['fert3']

<ipython-input-66-ecdf530dd22d>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
<ipython-input-66-ecdf530dd22d>:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['msp11_fert2'] = data_sample['msp11'] / data_sample['fert2']
<ipython-input-66-ecdf530dd22d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in t

In [67]:
# Step 8: Use rainfall variables month-wise (May to December)
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']

In [68]:
# Step 9: Use reservoir variables
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']

In [69]:
# Step 10: Use fortnightly temperature data from July to October
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max',
                    'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

In [70]:
# Step 11: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

In [71]:
# Step 12: Generate interaction terms for the different irrigation types
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for rainfall_var in rainfall_vars:
        data_sample[f'{rainfall_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[irrigation_type]
    for res_var in reservoir_vars:
        data_sample[f'{res_var}_{irrigation_type}'] = data_sample[res_var] * data_sample[irrigation_type]
    for rainfall_var in rainfall_vars:
        for res_var in reservoir_vars:
            data_sample[f'{rainfall_var}_{res_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[res_var]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]
    data_sample[f'groundwater_rainfall_{irrigation_type}'] = data_sample['groundwater'] * data_sample[rainfall_var]

<ipython-input-71-8c6aa7ff69a4>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_sample[f'{rainfall_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[irrigation_type]
<ipython-input-71-8c6aa7ff69a4>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead. To get a de-fragmented frame, use `newframe = frame.copy()`
  data_sample[f'{rainfall_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[irrigation_type]
<ipython-input-71-8c6aa7ff69a4>:4: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, whi

In [72]:
# Step 13: Define the initial set of independent variables
X_full = data_sample[['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3',
                      'groundwater'] + rainfall_vars + reservoir_vars + temperature_vars]

In [73]:
# Add all dummy variables for soil types (excluding one to avoid multicollinearity)
X_full = pd.concat([X_full, soil_dummies], axis=1)

# Add all irrigation interaction terms to the model
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for rainfall_var in rainfall_vars:
        X_full = pd.concat([X_full, data_sample[[f'{rainfall_var}_{irrigation_type}']]], axis=1)
    for res_var in reservoir_vars:
        X_full = pd.concat([X_full, data_sample[[f'{res_var}_{irrigation_type}']]], axis=1)
    for rainfall_var in rainfall_vars:
        for res_var in reservoir_vars:
            X_full = pd.concat([X_full, data_sample[[f'{rainfall_var}_{res_var}_{irrigation_type}']]], axis=1)

y = data_sample['yield']


In [77]:
print(data_sample.columns)


Index(['state', 'district', 'area', 'yield', 'prod', 'irrigation_nia',
       'irrigation_canal', 'irrigation_well', 'irrigation_tank',
       'irrigation_others',
       ...
       'rainfall12_res_2_irrigation_others',
       'rainfall12_res_3_irrigation_others',
       'rainfall12_res_4_irrigation_others',
       'rainfall12_res_5_irrigation_others',
       'rainfall12_res_6_irrigation_others',
       'rainfall12_res_7_irrigation_others',
       'rainfall12_res_8_irrigation_others',
       'rainfall12_res_9_irrigation_others', 'groundwater_irrigation_others',
       'groundwater_rainfall_irrigation_others'],
      dtype='object', length=518)


In [76]:
# Assuming 'stateid', 'districtid', and 'year' are your identifiers for panel data
data_sample.set_index(['stateid', 'districtid', 'year'], inplace=True)

# Ensure the independent (X) and dependent (y) variables are also MultiIndexed
X_full = data_sample[independent_vars]  # independent_vars is the list of your independent variables
y = data_sample['yield']  # yield is your dependent variable

# Check the first few rows of the DataFrame
print(X_full.head())
print(X_full.index)



KeyError: "None of ['stateid', 'districtid', 'year'] are in the columns"

In [75]:
# Step 14: Run Random Effects Model
re_model = RandomEffects(y, X_full)
re_results = re_model.fit()

ValueError: DataFrame input must have a MultiIndex with 2 levels

In [87]:
# Step 1: Install necessary packages
!pip install linearmodels

# Step 2: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

# Step 3: Set the file path (replace with your file path)
file_path = '/content/drive/My Drive/panel_data_2000_2024.csv'
data = pd.read_csv(file_path)
data.head()

# Step 4: Load the data (including independent variables for 2021 to 2024)
import pandas as pd
from linearmodels.panel import RandomEffects

data = pd.read_csv(file_path)

# Step 5: Set the panel index (stateid, districtid, year)
data = data.set_index(['stateid', 'districtid', 'year'])

# Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

# Step 6: Calculate the 1-year lag for the average price
data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level=['stateid', 'districtid']).shift(1)

# Step 7: Generate variables by dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
data_sample['msp11_fert2'] = data_sample['msp11'] / data_sample['fert2']
data_sample['msp11_fert3'] = data_sample['msp11'] / data_sample['fert3']

data_sample['avg_price_fert1'] = data_sample['avg_price_lag1'] / data_sample['fert1']
data_sample['avg_price_fert2'] = data_sample['avg_price_lag1'] / data_sample['fert2']
data_sample['avg_price_fert3'] = data_sample['avg_price_lag1'] / data_sample['fert3']

# Step 8: Use rainfall variables month-wise (May to December)
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']

# Step 9: Use reservoir variables
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']

# Step 10: Use fortnightly temperature data from July to October
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max',
                    'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

# Step 11: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

# Step 12: Generate interaction terms for the different irrigation types
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for rainfall_var in rainfall_vars:
        data_sample[f'{rainfall_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[irrigation_type]
    for res_var in reservoir_vars:
        data_sample[f'{res_var}_{irrigation_type}'] = data_sample[res_var] * data_sample[irrigation_type]
    for rainfall_var in rainfall_vars:
        for res_var in reservoir_vars:
            data_sample[f'{rainfall_var}_{res_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[res_var]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]
    data_sample[f'groundwater_rainfall_{irrigation_type}'] = data_sample['groundwater'] * data_sample[rainfall_var]

# Step 13: Define the initial set of independent variables
X_full = data_sample[['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3',
                      'groundwater'] + rainfall_vars + reservoir_vars + temperature_vars]

# Add all dummy variables for soil types (excluding one to avoid multicollinearity)
X_full = pd.concat([X_full, soil_dummies], axis=1)

# Add all irrigation interaction terms to the model
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for rainfall_var in rainfall_vars:
        X_full = pd.concat([X_full, data_sample[[f'{rainfall_var}_{irrigation_type}']]], axis=1)
    for res_var in reservoir_vars:
        X_full = pd.concat([X_full, data_sample[[f'{res_var}_{irrigation_type}']]], axis=1)
    for rainfall_var in rainfall_vars:
        for res_var in reservoir_vars:
            X_full = pd.concat([X_full, data_sample[[f'{rainfall_var}_{res_var}_{irrigation_type}']]], axis=1)

y = data_sample['yield']

# Step 14: Run Random Effects Model
re_model = RandomEffects(y, X_full)
re_results = re_model.fit()

# Step 15: Select only positively significant variables
significant_vars = re_results.pvalues[re_results.pvalues < 0.05].index
positive_vars = re_results.params[re_results.params > 0].index
significant_positive_vars = list(set(significant_vars).intersection(set(positive_vars)))

print("Significant positive variables:", significant_positive_vars)

# Step 16: Run regression again with only the significant positive variables
X_significant = data_sample[significant_positive_vars]
re_model_significant = RandomEffects(y, X_significant)
re_results_significant = re_model_significant.fit()

print(re_results_significant.summary)

# Step 17: Forecast yield for 2021 to 2024 using the significant positive variables

# Filter the data for the forecast period (2021 to 2024)
forecast_data = data.loc[(data.index.get_level_values('year') >= 2021) & (data.index.get_level_values('year') <= 2024)]

# Use the significant variables for the forecast period
X_forecast = forecast_data[significant_positive_vars]

# Step 18: Predict yields for 2021 to 2024
forecast_data['predicted_yield'] = re_results_significant.predict(X_forecast)

# Step 19: Display the forecasted results
print(forecast_data[['stateid', 'districtid', 'year', 'predicted_yield']])

# Optional: Save forecasted results to a file
forecast_data[['stateid', 'districtid', 'year', 'predicted_yield']].to_csv("/content/drive/My Drive/forecasted_yields_2021_2024.csv", index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-87-a0f080b5516d>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level=['stateid', 'districtid']).shift(1)
<ipython-input-87-a0f080b5516d>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
<ipython-input-87-a0f080b5516d>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice

ValueError: DataFrame input must have a MultiIndex with 2 levels

In [93]:
# Step 1: Install necessary packages
!pip install linearmodels statsmodels

# Step 2: Import libraries
import pandas as pd
from linearmodels.panel import RandomEffects
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

# Step 3: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

# Step 4: Set the file path and load the data
file_path = '/content/drive/My Drive/panel_data_2000_2024.csv'
data = pd.read_csv(file_path)

# Step 5: Declare panel data by creating 'panel_id' as a combination of 'stateid' and 'districtid'
data['panel_id'] = data['stateid'].astype(str) + '_' + data['districtid'].astype(str)
data = data.set_index(['panel_id', 'year'])

# Step 6: Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

# Step 7: Calculate the 1-year lag for the average price
data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)

# Step 8: Generate variables by dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
data_sample['msp11_fert2'] = data_sample['msp11'] / data_sample['fert2']
data_sample['msp11_fert3'] = data_sample['msp11'] / data_sample['fert3']

data_sample['avg_price_fert1'] = data_sample['avg_price_lag1'] / data_sample['fert1']
data_sample['avg_price_fert2'] = data_sample['avg_price_lag1'] / data_sample['fert2']
data_sample['avg_price_fert3'] = data_sample['avg_price_lag1'] / data_sample['fert3']

# Step 9: Use rainfall variables month-wise (May to December)
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']

# Step 10: Use reservoir variables
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']

# Step 11: Use fortnightly temperature data from July to October
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max',
                    'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

# Step 12: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

# Step 13: Generate interaction terms for different irrigation types
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for rainfall_var in rainfall_vars:
        data_sample[f'{rainfall_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[irrigation_type]
    for res_var in reservoir_vars:
        data_sample[f'{res_var}_{irrigation_type}'] = data_sample[res_var] * data_sample[irrigation_type]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]

# Step 14: Define the initial set of independent variables
X_full = data_sample[['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3',
                      'groundwater'] + rainfall_vars + reservoir_vars + temperature_vars]

# Add soil type dummies to X_full
X_full = pd.concat([X_full, soil_dummies], axis=1)

# Step 15: Ensure that X_full contains only numeric data
X_full = X_full.apply(pd.to_numeric, errors='coerce')

# Step 16: Drop rows with missing values in X_full
X_full = X_full.dropna()

# Step 17: Check for multicollinearity by calculating VIF
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

vif_df = calculate_vif(X_full)
print("VIF Data:\n", vif_df)

# Step 18: Drop variables with high VIF (typically VIF > 10) to avoid multicollinearity
high_vif_vars = vif_df[vif_df["VIF"] > 10]["Variable"]
X_full = X_full.drop(columns=high_vif_vars)

# Step 19: Define dependent variable 'y'
y = data_sample['yield']

# Step 20: Run Random Effects Model
re_model = RandomEffects(y, X_full)
re_results = re_model.fit()

print(re_results.summary())

# Step 21: Select only significant variables from price-related section
significant_vars = re_results.pvalues[re_results.pvalues < 0.05].index
positive_vars = re_results.params[re_results.params > 0].index
significant_positive_vars = list(set(significant_vars).intersection(set(positive_vars)))

print("Significant positive variables:", significant_positive_vars)

# Step 22: Run regression again with only the significant positive variables from the price section
X_significant = X_full[significant_positive_vars]
re_model_significant = RandomEffects(y, X_significant)
re_results_significant = re_model_significant.fit()

print(re_results_significant.summary())

# Step 23: Forecast yield for 2021 to 2024 using significant positive variables
forecast_data = data.loc[(data.index.get_level_values('year') >= 2021) & (data.index.get_level_values('year') <= 2024)]

# Use the significant variables for the forecast period
X_forecast = forecast_data[significant_positive_vars]

# Step 24: Predict yields for 2021 to 2024
forecast_data['predicted_yield'] = re_results_significant.predict(X_forecast)

# Step 25: Display forecasted results
print(forecast_data[['panel_id', 'year', 'predicted_yield']])

# Optional: Save forecasted results to a CSV file
forecast_data[['panel_id', 'year', 'predicted_yield']].to_csv("/content/drive/My Drive/forecasted_yields_2021_2024.csv", index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-93-4abf102772e9>:27: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)
<ipython-input-93-4abf102772e9>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
<ipython-input-93-4abf102772e9>:31: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''

In [96]:
# Step 1: Install necessary packages
!pip install linearmodels statsmodels

# Step 2: Import libraries
import pandas as pd
from linearmodels.panel import RandomEffects
from statsmodels.stats.outliers_influence import variance_inflation_factor
import numpy as np

# Step 3: Mount Google Drive (if using Google Drive for data storage)
from google.colab import drive
drive.mount('/content/drive')

# Step 4: Set the file path and load the data
file_path = '/content/drive/My Drive/panel_data_2000_2024.csv'
data = pd.read_csv(file_path)

# Step 5: Declare panel data by creating 'panel_id' as a combination of 'stateid' and 'districtid'
data['panel_id'] = data['stateid'].astype(str) + '_' + data['districtid'].astype(str)
data = data.set_index(['panel_id', 'year'])

# Step 6: Filter the data for the sample period (2000 to 2020)
data_sample = data.loc[data.index.get_level_values('year') <= 2020]

# Step 7: Calculate the 1-year lag for the average price
data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)

# Step 8: Generate variables by dividing msp11 and the lagged avg_price by fert1, fert2, and fert3
data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
data_sample['msp11_fert2'] = data_sample['msp11'] / data_sample['fert2']
data_sample['msp11_fert3'] = data_sample['msp11'] / data_sample['fert3']

data_sample['avg_price_fert1'] = data_sample['avg_price_lag1'] / data_sample['fert1']
data_sample['avg_price_fert2'] = data_sample['avg_price_lag1'] / data_sample['fert2']
data_sample['avg_price_fert3'] = data_sample['avg_price_lag1'] / data_sample['fert3']

# Step 9: Use rainfall variables month-wise (May to December)
rainfall_vars = ['rainfall5', 'rainfall6', 'rainfall7', 'rainfall8', 'rainfall9', 'rainfall10', 'rainfall11', 'rainfall12']

# Step 10: Use reservoir variables
reservoir_vars = ['res_1', 'res_2', 'res_3', 'res_4', 'res_5', 'res_6', 'res_7', 'res_8', 'res_9']

# Step 11: Use fortnightly temperature data from July to October
temperature_vars = ['july1_max', 'aug1_max', 'sep1_max', 'oct1_max', 'aug2_max', 'sep2_max', 'oct2_max',
                    'july1_min', 'aug1_min', 'sep1_min', 'oct1_min', 'july2_min', 'aug2_min', 'sep2_min', 'oct2_min']

# Step 12: Convert soil_type into dummy variables
soil_dummies = pd.get_dummies(data_sample['soil_type'], prefix='soil', drop_first=True)
data_sample = pd.concat([data_sample, soil_dummies], axis=1)

# Step 13: Generate interaction terms for different irrigation types
for irrigation_type in ['irrigation_nia', 'irrigation_canal', 'irrigation_well', 'irrigation_tank', 'irrigation_others']:
    for rainfall_var in rainfall_vars:
        data_sample[f'{rainfall_var}_{irrigation_type}'] = data_sample[rainfall_var] * data_sample[irrigation_type]
    for res_var in reservoir_vars:
        data_sample[f'{res_var}_{irrigation_type}'] = data_sample[res_var] * data_sample[irrigation_type]
    data_sample[f'groundwater_{irrigation_type}'] = data_sample['groundwater'] * data_sample[irrigation_type]

# Step 14: Define the initial set of independent variables
X_full = data_sample[['msp11_fert1', 'msp11_fert2', 'msp11_fert3', 'avg_price_fert1', 'avg_price_fert2', 'avg_price_fert3',
                      'groundwater'] + rainfall_vars + reservoir_vars + temperature_vars]

# Add soil type dummies to X_full
X_full = pd.concat([X_full, soil_dummies], axis=1)

# Step 15: Ensure that X_full contains only numeric data
X_full = X_full.apply(pd.to_numeric, errors='coerce')

# Step 16: Drop rows with missing values in X_full
X_full = X_full.dropna()

# Step 17: Handle infinite values by replacing them with NaN
X_full.replace([np.inf, -np.inf], np.nan, inplace=True)
X_full = X_full.dropna()

# Step 18: Check for multicollinearity by calculating VIF
def calculate_vif(X):
    vif_data = pd.DataFrame()
    vif_data["Variable"] = X.columns
    # Ensure X is numeric and no NaN values exist
    X = X.fillna(0)  # Replace NaN with 0 or another suitable value
    vif_data["VIF"] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
    return vif_data

vif_df = calculate_vif(X_full)
print("VIF Data:\n", vif_df)

# Step 19: Drop variables with high VIF (typically VIF > 10) to avoid multicollinearity
high_vif_vars = vif_df[vif_df["VIF"] > 10]["Variable"]
X_full = X_full.drop(columns=high_vif_vars)

# Step 20: Define dependent variable 'y'
y = data_sample['yield']

# Step 21: Run Random Effects Model
re_model = RandomEffects(y, X_full)
re_results = re_model.fit()

print(re_results.summary())

# Step 22: Select only significant variables from price-related section
significant_vars = re_results.pvalues[re_results.pvalues < 0.05].index
positive_vars = re_results.params[re_results.params > 0].index
significant_positive_vars = list(set(significant_vars).intersection(set(positive_vars)))

print("Significant positive variables:", significant_positive_vars)

# Step 23: Run regression again with only the significant positive variables from the price section
X_significant = X_full[significant_positive_vars]
re_model_significant = RandomEffects(y, X_significant)
re_results_significant = re_model_significant.fit()

print(re_results_significant.summary())

# Step 24: Forecast yield for 2021 to 2024 using significant positive variables
forecast_data = data.loc[(data.index.get_level_values('year') >= 2021) & (data.index.get_level_values('year') <= 2024)]

# Use the significant variables for the forecast period
X_forecast = forecast_data[significant_positive_vars]

# Step 25: Predict yields for 2021 to 2024
forecast_data['predicted_yield'] = re_results_significant.predict(X_forecast)

# Step 26: Display forecasted results
print(forecast_data[['panel_id', 'year', 'predicted_yield']])

# Optional: Save forecasted results to a CSV file
forecast_data[['panel_id', 'year', 'predicted_yield']].to_csv("/content/drive/My Drive/forecasted_yields_2021_2024.csv", index=False)


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-96-2fdc3597d742>:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['avg_price_lag1'] = data_sample[['price_crop10m11', 'price_crop11m11', 'price_crop12m11']].mean(axis=1).groupby(level='panel_id').shift(1)
<ipython-input-96-2fdc3597d742>:29: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_sample['msp11_fert1'] = data_sample['msp11'] / data_sample['fert1']
<ipython-input-96-2fdc3597d742>:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFra

TypeError: ufunc 'isfinite' not supported for the input types, and the inputs could not be safely coerced to any supported types according to the casting rule ''safe''